In [29]:
import json
from tabulate import tabulate

# Función para separar la lógica de carga y procesamiento de datos del resto del código.
def cargar_datos(ruta):
    # Leer/importar el archivo tipo JSON y se carga en la variable
    with open(ruta, 'r', encoding='utf-8') as archivo:
        datos = json.load(archivo)
    
    # Definir los encabezados que se asignarán a cada columna
    encabezados = ['Identificacion', 'Telefono', 'Provincia', 'Canton', 'Enfermedad', 'Medicamento']
    
    # Este paso se realiza, según se investigó, para convertir las listas en diccionarios (cuando sea necesario)
    if isinstance(datos, list) and all(isinstance(fila, list) for fila in datos):
        datos = [dict(zip(encabezados, fila)) for fila in datos]

    # Convertir la primera letra de cada valor de cada columna de texto y corregir el nombre de la provincia San José
    for dato in datos:
        dato['Provincia'] = dato['Provincia'].title().replace('San Jose', 'San José')
        dato['Canton'] = dato['Canton'].title()
        dato['Enfermedad'] = dato['Enfermedad'].title()
        dato['Medicamento'] = dato['Medicamento'].title()

    # Ordenar los datos por Identificacion
    return sorted(datos, key=lambda x: x['Identificacion']) # le dice a sorted qué atributo usar para la ordenación, en este caso Identificación

# Cargar datos y ejecutar la búsqueda
datos_procesados = cargar_datos('C:/Users/Kevin Esquivel/Desktop/Cursos Python/01. Python Básico/Semana III/Clase 05/clinica_intermedio.json')

# Función para solicitar la identificación que desea consulta el usuario y mostrar los datos relacionados
def busqueda_identificacion(datos):
    while True:
        try:
            identificacion = input('Por favor, ingrese la identificación que desea buscar: ')
            break
        except ValueError:
            print('\nError: La identificación debe ser un número, no texto. Por favor intente de nuevo.')
    
    resultados = [d for d in datos if str(d['Identificacion']) == identificacion]

    if resultados:
        print(f'\nA continuación, se detalla la información encontrada de la identicación {identificacion}:')
        print(tabulate([list(d.values()) for d in resultados], headers=datos[0].keys(), tablefmt='grid'))
        # Notas importantes:
        # Existen tres tipos de visualización de la tabla: 
        # grid (diseño con guiones para cada fila y columna). 
        # pipe (mismo que el anterior, pero sin guiones alrededor).
        # plain (plano, sin líneas ni guiones, es decir, sin diseño).
    else:
        print(f'\nNo se encontró información para la identificación ingresada: {identificacion}.')

def reporte_enfermedades(datos):
    enfermedades_contadas = {}

    for dato in datos:
        enfermedad = dato['Enfermedad']
        if enfermedad in enfermedades_contadas:
            enfermedades_contadas[enfermedad] += 1
        else:
            enfermedades_contadas[enfermedad] = 1

    enfermedades_ordenadas = sorted(enfermedades_contadas.items(), key=lambda x: x[1], reverse=True)
    
    print('\n\nReporte de enfermedades tratadas en la clínica:')
    print(tabulate(enfermedades_ordenadas, headers=['Enfermedad', 'Cantidad de Pacientes'], tablefmt='grid'))
    
def reporte_medicamentos(datos):
    medicamentos_contados = {}

    for dato in datos:
        medicamento = dato['Medicamento']
        if medicamento in medicamentos_contados:
            medicamentos_contados[medicamento] += 1
        else:
            medicamentos_contados[medicamento] = 1

    medicamentos_ordenados = sorted(medicamentos_contados.items(), key=lambda x: x[1], reverse=True)
    
    print("\n\nReporte de medicamentos recetados en la clínica:")
    print(tabulate(medicamentos_ordenados, headers=["Medicamento", "Cantidad de Pacientes"], tablefmt="grid"))
    
def reporte_provincias(datos):
    provincias_contadas = {}

    for dato in datos:
        provincia = dato['Provincia']
        if provincia in provincias_contadas:
            provincias_contadas[provincia] += 1
        else:
            provincias_contadas[provincia] = 1

    provincias_ordenadas = sorted(provincias_contadas.items(), key=lambda x: x[1], reverse=True)
    
    print("\n\nReporte de provincias con más pacientes la clínica:")
    print(tabulate(provincias_ordenadas, headers=["Provincia", "Cantidad de Pacientes"], tablefmt="grid"))

def comparar_pacientes(datos):
    identificacion1 = input('Por favor ingrese la identificación del primer paciente: ')
    identificacion2 = input('Por favor ingrese la identificación del segundo paciente: ')

    paciente1 = next((d for d in datos if str(d['Identificacion']) == str(identificacion1)), None)
    paciente2 = next((d for d in datos if str(d['Identificacion']) == str(identificacion2)), None)

    if not paciente1 or not paciente2:
        print("\nError: Una o ambas identificaciones de los pacientes ingresados no fueron encontradas en la base de datos.")
        return
    
    enfermedades_comunes = set([paciente1['Enfermedad']]) & set([paciente2['Enfermedad']])
    medicamentos_comunes = set([paciente1['Medicamento']]) & set([paciente2['Medicamento']])

    print(f'\n\nReporte comparativo entre el primer paciente con identificación {identificacion1} y el segundo paciente con identificación {identificacion2}:')

    if enfermedades_comunes:
        print('\nEnfermedades en común:')
        print(tabulate([[e] for e in enfermedades_comunes], tablefmt='grid'))
    else:
        print('\nNo tienen enfermedades en común.')

    if medicamentos_comunes:
        print('\nMedicamentos en común:')
        print(tabulate([[m] for m in medicamentos_comunes], tablefmt='grid'))
    else:
        print('\nNo tienen medicamentos en común.')

while True:
    print("\n¡Bienvenido al sistema en línea del Hospital Max Peralta!")
    print("\nMenú de opciones:")
    print("1. Información de paciente por identificación")
    print("2. Reporte de enfermedades")
    print("3. Reporte de medicamentos")
    print("4. Reporte de provincias")
    print("5. Comparativo de dos pacientes")
    print("6. Salir")
    
    opcion = input("Por favor, seleccione una opción del menú anterior: ")
    
    if opcion == '1':
        busqueda_identificacion(datos_procesados)
    elif opcion == '2':
        reporte_enfermedades(datos_procesados)
    elif opcion == '3':
        reporte_medicamentos(datos_procesados)
    elif opcion == '4':
        reporte_provincias(datos_procesados)
    elif opcion == '5':
        comparar_pacientes(datos_procesados)
    elif opcion == '6':
        print("\n¡Gracias por utilizar nuestro programa, nos vemos pronto!")
        break
    else:
        print("Opción no válida, por favor intente de nuevo.")


## Para pruebas de funcionamiento
#busqueda_identificacion(datos_procesados)
#reporte_enfermedades(datos_procesados)
#reporte_medicamentos(datos_procesados)
#reporte_provincias(datos_procesados)
#comparar_pacientes(datos_procesados)
#print(datos_procesados)


¡Bienvenido al sistema en línea del Hospital Max Peralta!

Menú de opciones:
1. Información de paciente por identificación
2. Reporte de enfermedades
3. Reporte de medicamentos
4. Reporte de provincias
5. Comparativo de dos pacientes
6. Salir

No se encontró información para la identificación ingresada: 123.

¡Bienvenido al sistema en línea del Hospital Max Peralta!

Menú de opciones:
1. Información de paciente por identificación
2. Reporte de enfermedades
3. Reporte de medicamentos
4. Reporte de provincias
5. Comparativo de dos pacientes
6. Salir


Reporte comparativo entre el primer paciente con identificación 104598273 y el segundo paciente con identificación 116310912:

No tienen enfermedades en común.

Medicamentos en común:
+--------------+
| Acetaminofen |
+--------------+

¡Bienvenido al sistema en línea del Hospital Max Peralta!

Menú de opciones:
1. Información de paciente por identificación
2. Reporte de enfermedades
3. Reporte de medicamentos
4. Reporte de provincias
5. Co